<div style="border-radius:20px;
            border : black solid;
            background-color: ##FFFFFF;
            font-size:200%;
            text-align: left">

<h1 style='; border:0; border-radius: 15px; text-shadow: 1px 1px black; font-weight: bold; color:green'><center> Disaster Tweet Classification using NLP </center></h1>

<div style='font-size:200%;'>
    <a id='import'></a>
    <h1 style='color: green; font-weight: bold; font-family: Cascadia code;'>
        <center> Importing libraries</center>
    </h1>
  
</div>

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
from collections import defaultdict


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
import os
for dirname, _, filenames in os.walk('/Python MP/Dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/Python MP/Dataset\test.csv
/Python MP/Dataset\train.csv


In [8]:
Train = pd.read_csv('/Python MP/Dataset/train.csv')
Test = pd.read_csv('/Python MP/Dataset/test.csv')
Test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [5]:
Train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


<h1 style='; border:0; border-radius: 15px; text-shadow: 1px 1px black; font-weight: bold; color:black'><center>  Data Pre-processing ⌛</center></h1> <a id='preprocess'></a>

<h2 align="center" ><a id='re'><b>Removing unnecessary characters and special symbols<b></a></h2>

In [9]:
def cleanText(text):
    whitespace = re.compile(r"\s+")
    web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
    user = re.compile(r"(?i)@[a-z0-9_]+")
    text = whitespace.sub(' ', text)
    text = web_address.sub('', text)
    text = user.sub('', text)
    text = re.sub(r"\[[^()]*\]", "", text)
    text = re.sub("\d+", "", text)
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r"(?:@\S*|#\S*|http(?=.*://)\S*)", "", text)
    return text.lower()

Train.text = [cleanText(item) for item in Train.text]
Train

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,all residents asked to shelter in place are be...,1
3,6,NaN,NaN,people receive wildfires evacuation orders in...,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,the out of control wild fires in california ...,1
7610,10871,NaN,NaN,m km s of volcano hawaii httptcozdtoydebj,1
7611,10872,NaN,NaN,police investigating after an ebike collided w...,1


In [10]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)
        
text = "corect me plese"
correct_spellings(text)

'correct me plese'

<h2 align="center" ><a id='fit'><b>Fitting tokenizer on texts<b></a></h2>

In [11]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.oov_token = '<oovToken>'
tokenizer.fit_on_texts(Train.text)
vocab = tokenizer.word_index
vocabCount = len(vocab)+1

vocabCount

19198

In [12]:
x_Train = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(Train.text.to_numpy()), padding='pre')
y_Train = Train.target.to_numpy()

x_Train.shape, y_Train.shape

((7613, 31), (7613,))

<h2 style='; border:0; border-radius: 15px; text-shadow: 1px 1px black; font-weight: bold; color:black'><center> Model Creation and Evalutation </center></h2><a id="model"></a>

## **Sequential model**

#### A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.

The Embedding layer takes the integer-encoded vocabulary and looks up the embedding vector for each word-index. 

Global Average Pooling is a pooling operation designed to replace fully connected layers in classical CNNs. The idea is to generate one feature map for each corresponding category of the classification task in the last mlpconv layer. Instead of adding fully connected layers on top of the feature maps, we take the average of each feature map.

A dense layer is a layer that is deeply connected with its preceding layer which means the neurons of the layer are connected to every neuron of its preceding layer. This layer is the most commonly used layer in artificial neural network networks.

In [70]:
from tensorflow.keras import regularizers

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=vocabCount+1, output_dim=64, input_length=31))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 31, 64)            1228736   
                                                                 
 global_average_pooling1d_1   (None, 64)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,235,009
Trainable params: 1,235,009
Non-trainable params: 0
____________________________________________

In [71]:
model.fit(x_Train, y_Train, epochs=10, shuffle=True)

Epoch 1/10
238/238 [==============================] - 2s 5ms/step - loss: 0.9773 - accuracy: 0.5696
Epoch 2/10
238/238 [==============================] - 1s 5ms/step - loss: 0.5645 - accuracy: 0.7850
Epoch 3/10
238/238 [==============================] - 1s 5ms/step - loss: 0.4208 - accuracy: 0.8719
Epoch 4/10
238/238 [==============================] - 1s 5ms/step - loss: 0.3487 - accuracy: 0.9052
Epoch 5/10
238/238 [==============================] - 1s 5ms/step - loss: 0.2976 - accuracy: 0.9280
Epoch 6/10
238/238 [==============================] - 1s 5ms/step - loss: 0.2578 - accuracy: 0.9433
Epoch 7/10
238/238 [==============================] - 1s 5ms/step - loss: 0.2287 - accuracy: 0.9560
Epoch 8/10
238/238 [==============================] - 1s 5ms/step - loss: 0.2063 - accuracy: 0.9647
Epoch 9/10
238/238 [==============================] - 1s 4ms/step - loss: 0.1862 - accuracy: 0.9715
Epoch 10/10
238/238 [==============================] - 1s 5ms/step - loss: 0.1722 - accuracy: 0.9765

In [72]:
model.save('Tweet.h5')

In [5]:
model = tf.keras.models.load_model('\Python MP\Tweet.h5')

In [11]:
Test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [12]:
idCol = Test['id'].to_numpy()
Test = Test.drop(columns=['keyword', 'location', 'id'])
Test.head()

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan


In [13]:
Test.text = [cleanText(item) for item in Test.text]
Test

,text
0,just happened a terrible car crash
1,heard about earthquake is different cities sta...
2,there is a forest fire at spot pond geese are ...
3,apocalypse lighting spokane wildfires
4,typhoon soudelor kills in china and taiwan
...,...
3258,earthquake safety los angeles ûò safety fasten...
3259,storm in ri worse than last hurricane my citya...
3260,green line derailment in chicago httptcoutbxlc...
3261,meg issues hazardous weather outlook hwo httpt...


In [14]:
x_Test = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(Test.text.to_numpy()), padding='pre', maxlen=31)

x_Test.shape


(3263, 31)

In [15]:
preds = model.predict(x_Test)
preds

102/102 [==============================] - 4s 1ms/step


array([[0.7875675],
       [0.6446478],
       [0.9928756],
       ...,
       [0.9441219],
       [0.9479781],
       [0.8513665]], dtype=float32)

In [16]:
preds = preds.reshape(len(preds))
for i in range(len(preds)):
    if preds[i]>0.9:
        preds[i] = 1
    else:
        preds[i] = 0

submission = pd.DataFrame({'id': idCol, 'target': preds})
submission.target = submission.target.astype(int)
submission.set_index('id')

,target
id,
0,0
2,0
3,1
9,0
11,1
...,...
10861,0
10865,1
10868,1


In [13]:
user_input = input("Enter tweet : ")
cleaned_input = cleanText(user_input)
tokenized_input = tokenizer.texts_to_sequences([cleaned_input])
padded_input = tf.keras.preprocessing.sequence.pad_sequences(tokenized_input, padding='pre', maxlen=31)
prediction = model.predict(padded_input)
prediction
if(prediction > 0.7):
    print("Disaster")
else:
    print("Not a disaster, chill out!")

Enter tweet : Earthquake in rajasthan
1/1 [==============================] - 1s 1s/step
Disaster
